In [ ]:
!pip install tensorboardX

In [ ]:
#pip uninstall imgaug

Found existing installation: imgaug 0.4.0
Uninstalling imgaug-0.4.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/imgaug-0.4.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/imgaug/*
Proceed (y/n)? n


In [ ]:
import argparse
#from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import glob
from torchvision import transforms
from PIL import Image
import numpy as np
from imgaug import augmenters as iaa
import imgaug as ia
import socket
from datetime import datetime
import os
import torch
from tensorboardX import SummaryWriter
import tqdm
import torch.nn as nn
from torch.nn import functional as F
import math
import torch.backends.cudnn as cudnn
from torch.nn import init


Requirement already satisfied: imgaug in /usr/local/lib/python3.7/dist-packages (0.4.0)
Requirement already satisfied: Shapely in /usr/local/lib/python3.7/dist-packages (from imgaug) (1.8.0)
Requirement already satisfied: imageio in /usr/local/lib/python3.7/dist-packages (from imgaug) (2.4.1)
Requirement already satisfied: matplotlib in /usr/local/lib/python3.7/dist-packages (from imgaug) (3.2.2)
Requirement already satisfied: opencv-python in /usr/local/lib/python3.7/dist-packages (from imgaug) (4.1.2.30)
Requirement already satisfied: Pillow in /usr/local/lib/python3.7/dist-packages (from imgaug) (7.1.2)
Requirement already satisfied: scipy in /usr/local/lib/python3.7/dist-packages (from imgaug) (1.4.1)
Requirement already satisfied: six in /usr/local/lib/python3.7/dist-packages (from imgaug) (1.15.0)
Requirement already satisfied: scikit-image>=0.14.2 in /usr/local/lib/python3.7/dist-packages (from imgaug) (0.18.3)
Requirement already satisfied: numpy>=1.15 in /usr/local/lib/python3.7/dist-packages (from imgaug) (1.19.5)
Requirement already satisfied: tifffile>=2019.7.26 in /usr/local/lib/python3.7/dist-packages (from scikit-image>=0.14.2->imgaug) (2021.11.2)
Requirement already satisfied: PyWavelets>=1.1.1 in /usr/local/lib/python3.7/dist-packages (from scikit-image>=0.14.2->imgaug) (1.2.0)
Requirement already satisfied: networkx>=2.0 in /usr/local/lib/python3.7/dist-packages (from scikit-image>=0.14.2->imgaug) (2.6.3)
Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.7/dist-packages (from matplotlib->imgaug) (0.11.0)
Requirement already satisfied: python-dateutil>=2.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib->imgaug) (2.8.2)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib->imgaug) (2.4.7)
Requirement already satisfied: kiwisolver>=1.0.1 in /usr/local/lib/python3.7/dist-packages (from matplotlib->imgaug) (1.3.2)

In [ ]:
ROOT_path = os.path.abspath('../../../')



class DefaultConfig(object):
    num_epochs=1 #pre10
    epoch_start_i=0
    checkpoint_step=5
    validation_step = 1
    crop_height = 256
    crop_width = 448
    batch_size = 2
    #训练集所在位置，根据自身训练集位置进行修改
    #data = r'train_dataset/'
    data = r'/content/drive/MyDrive/Blood/train_dataset'

    log_dirs = os.path.join(ROOT_path, 'Log/OCT')
    
    lr=0.01
    lr_mode= 'poly'
    net_work= 'BaseNet'
    #net_work= 'MSSeg'  #net_work= 'UNet'

    momentum = 0.9#
    weight_decay = 1e-4#


    mode='train'
    num_classes = 2

    
    k_fold=4
    test_fold=4
    num_workers=0
    
    cuda='0'
    use_gpu=True
    pretrained_model_path= os.path.join(ROOT_path, 'pretrained', 'resnet34-333f7ec4.pth')
    #save_model_path='./checkpoints'
    save_model_path='/content/drive/MyDrive/Blood/model/temp_model'

In [ ]:
from torch.autograd import Variable


class DiceLoss(nn.Module):
    def __init__(self,smooth=0.01):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
        
    def forward(self,input, target):
        input = torch.sigmoid(input)
        Dice = Variable(torch.Tensor([0]).float()).cuda()
        intersect=(input*target).sum()
        union = torch.sum(input) + torch.sum(target)
        Dice=(2*intersect+self.smooth)/(union+self.smooth)
        dice_loss=1-Dice
        return dice_loss

class Multi_DiceLoss(nn.Module):
    def __init__(self, class_num=5,smooth=0.001):
        super(Multi_DiceLoss, self).__init__()
        self.smooth = smooth
        self.class_num = class_num

    def forward(self,input, target):
        input = torch.exp(input)
        Dice = Variable(torch.Tensor([0]).float()).cuda()
        for i in range(0,self.class_num):
            input_i = input[:,i,:,:]
            target_i = (target == i).float()
            intersect = (input_i*target_i).sum()
            union = torch.sum(input_i) + torch.sum(target_i)
            dice = (2 * intersect + self.smooth) / (union + self.smooth)
            Dice += dice
        dice_loss = 1 - Dice/(self.class_num)
        return dice_loss

class EL_DiceLoss(nn.Module):
    def __init__(self, class_num=4,smooth=1,gamma=0.5):
        super(EL_DiceLoss, self).__init__()
        self.smooth = smooth
        self.class_num = class_num
        self.gamma = gamma

    def forward(self,input, target):
        input = torch.exp(input)
        self.smooth = 0.
        Dice = Variable(torch.Tensor([0]).float()).cuda()
        for i in range(1,self.class_num):
            input_i = input[:,i,:,:]
            target_i = (target == i).float()
            intersect = (input_i*target_i).sum()
            union = torch.sum(input_i) + torch.sum(target_i)
            if target_i.sum() == 0:
                dice = Variable(torch.Tensor([1]).float()).cuda()
            else:
                dice = (2 * intersect + self.smooth) / (union + self.smooth)
            Dice += (-torch.log(dice))**self.gamma
        dice_loss = Dice/(self.class_num - 1)
        return dice_loss

In [ ]:
def augmentation():
    # augment images with spatial transformation: Flip, Affine, Rotation, etc...
    # see https://github.com/aleju/imgaug for more details
    pass

def augmentation_pixel():
    # augment images with pixel intensity transformation: GaussianBlur, Multiply, etc...
    pass

class Data(torch.utils.data.Dataset):
    Unlabelled=[0, 0, 0]
    sick = [255, 255, 255]
    COLOR_DICT = np.array([Unlabelled,sick])
    def __init__(self, dataset_path,scale=(320, 320), mode='train'):
        super().__init__()
        self.mode = mode
        self.img_path=dataset_path+'/img'
        self.mask_path=dataset_path+'/mask'
        self.image_lists, self.label_lists=self.read_list(self.img_path)
        self.resize = scale
        self.flip =iaa.SomeOf((2,5),[     
             iaa.PiecewiseAffine(scale=(0, 0.1), nb_rows=4, nb_cols=4, cval=0),                  
             iaa.Fliplr(0.5),
             iaa.Flipud(0.1),         
             iaa.Affine(rotate=(-20, 20),
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}),
             iaa.OneOf([
                    iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(3, 5)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
             iaa.contrast.LinearContrast((0.5, 1.5))],
                              random_order=True)

        self.to_tensor = transforms.ToTensor()


    def __getitem__(self, index):
        # load image and crop
        img = Image.open(self.image_lists[index]).convert('RGB')
        img = img.resize(self.resize)
        img = np.array(img)
        labels=self.label_lists[index]
        #load label
        if self.mode !='test':
            label_ori = Image.open(self.label_lists[index]).convert('RGB')
            label_ori = label_ori.resize(self.resize)
            label_ori = np.array(label_ori)
            label=np.ones(shape=(label_ori.shape[0],label_ori.shape[1]),dtype=np.uint8)
            #print("****label size:")
            #print(label.shape)# 256*448
            #convert RGB  to one hot
            
            for i in range(len(self.COLOR_DICT)):
                equality = np.equal(label_ori, self.COLOR_DICT[i])
                class_map = np.all(equality, axis=-1)
                label[class_map]=i

            # augment image and label
            if self.mode == 'train':
                seq_det = self.flip.to_deterministic()#固定变换
                segmap = ia.SegmentationMapsOnImage(label, shape=label.shape)
                img = seq_det.augment_image(img)
                label = seq_det.augment_segmentation_maps([segmap])[0].get_arr().astype(np.uint8)

            label_img=torch.from_numpy(label.copy()).float()
            if self.mode == 'val':
                img_num=len(os.listdir(os.path.dirname(labels)))
                labels=label_img,img_num
            else:
                labels=label_img
        imgs=img.transpose(2, 0, 1)/255.0
        img = torch.from_numpy(imgs.copy()).float()#self.to_tensor(img.copy()).float()
        return img, labels

    def __len__(self):
        return len(self.image_lists)

    def read_list(self, image_path):
        fold = os.listdir(image_path)
        # fold = sorted(os.listdir(image_path), key=lambda x: int(x[-2:]))
        # print(fold)

        img_list = []
        label_list = []
        if self.mode=='train':
            for item in fold:
                name = item.split('.')[0]
                img_list.append(os.path.join(image_path, item))
                label_list.append(os.path.join(image_path.replace('img', 'mask'), '{}.png'.format(name)))


        elif self.mode == 'val':
            for item in fold:
                name = item.split('.')[0]
                img_list.append(os.path.join(image_path, item))
                label_list.append(os.path.join(image_path.replace('img', 'mask'), '{}.png'.format(name)))


        elif self.mode=='test':
            for item in fold:
                name = item.split('.')[0]
                img_list.append(os.path.join(image_path, item))
                label_list.append(os.path.join(image_path.replace('img', 'mask'), '{}.png'.format(name)))
                
        return img_list, label_list

In [ ]:
import shutil
import os.path as osp
def save_checkpoint(state,best_pred, epoch,is_best,checkpoint_path, filename='./checkpoint/checkpoint.pth'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, osp.join(checkpoint_path,'best.pth'))
    # shutil.copyfile(filename, osp.join(checkpoint_path, 'model_ep{}.pth'.format(epoch+1)))



def adjust_learning_rate(opt, optimizer, epoch):                        
    """
    Sets the learning rate to the initial LR decayed by 10 every 30 epochs(step = 30)
    """
    if opt.lr_mode == 'step':
        lr = opt.lr * (0.1 ** (epoch // opt.step))
    elif opt.lr_mode == 'poly':
        lr = opt.lr * (1 - epoch / opt.num_epochs) ** 0.9
    else:
        raise ValueError('Unknown lr mode {}'.format(opt.lr_mode))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr




def one_hot_it(label, label_info):
	# return semantic_map -> [H, W, num_classes]
	semantic_map = []
	for info in label_info:
		color = label_info[info]
		# colour_map = np.full((label.shape[0], label.shape[1], label.shape[2]), colour, dtype=int)
		equality = np.equal(label, color)
		class_map = np.all(equality, axis=-1)
		semantic_map.append(class_map)
	semantic_map = np.stack(semantic_map, axis=-1)
	return semantic_map
    
    
def compute_score(predict, target, forground = 1,smooth=1):
    score = 0
    count = 0
    target[target!=forground]=0
    predict[predict!=forground]=0
    assert(predict.shape == target.shape)
    overlap = ((predict == forground)*(target == forground)).sum() #TP
    union=(predict == forground).sum() + (target == forground).sum()-overlap #FP+FN+TP
    FP=(predict == forground).sum()-overlap #FP
    FN=(target == forground).sum()-overlap #FN
    TN= target.shape[0]*target.shape[1]-union #TN


    #print('overlap:',overlap)
    dice=(2*overlap +smooth)/ (union+overlap+smooth)
    
    precsion=((predict == target).sum()+smooth) / (target.shape[0]*target.shape[1]+smooth)
    
    jaccard=(overlap+smooth) / (union+smooth)

    Sensitivity=(overlap+smooth) / ((target == forground).sum()+smooth)

    Specificity=(TN+smooth) / (FP+TN+smooth)
    

    return dice,precsion,jaccard,Sensitivity,Specificity

def eval_multi_seg(predict, target,num_classes):
    # pred_seg=torch.argmax(torch.exp(predict),dim=1).int()
    pred_seg = predict.data.cpu().numpy()
    label_seg = target.data.cpu().numpy().astype(dtype=np.int)
    assert(pred_seg.shape == label_seg.shape)
    acc=(pred_seg==label_seg).sum()/(pred_seg.shape[0]*pred_seg.shape[1]*pred_seg.shape[2])
    
    # Dice = []
    # Precsion = []
    # Jaccard = []
    # Sensitivity=[]
    # Specificity=[]

    # n = pred_seg.shape[0]
    Dice=[]
    True_label=[]
    TP = FPN = 0
    for classes in range(1,num_classes):
        overlap=((pred_seg==classes)*(label_seg==classes)).sum()
        union=(pred_seg==classes).sum()+(label_seg==classes).sum()
        Dice.append((2*overlap+0.1)/(union+0.1))
        True_label.append((label_seg==classes).sum())
        TP += overlap
        FPN +=union

    
    return Dice,True_label,acc, 2*TP/(FPN+1)



    


    # for i in range(n):
    #     dice,precsion,jaccard,sensitivity,specificity= compute_score(pred_seg[i],label_seg[i])
    #     Dice.append(dice)
    #     Precsion .append(precsion)
    #     Jaccard.append(jaccard)
    #     Sensitivity.append(sensitivity)
    #     Specificity.append(specificity)

    # return Dice,Precsion,Jaccard,Sensitivity,Specificity


def eval_seg(predict, target, forground = 1):
    pred_seg=torch.round(torch.sigmoid(predict)).int()
    pred_seg = pred_seg.data.cpu().numpy()
    label_seg = target.data.cpu().numpy().astype(dtype=np.int)
    assert(pred_seg.shape == label_seg.shape)

    Dice = []
    Precsion = []
    Jaccard = []
    n = pred_seg.shape[0]
    
    for i in range(n):
        dice,precsion,jaccard = compute_score(pred_seg[i],label_seg[i])
        Dice .append(dice)
        Precsion .append(precsion)
        Jaccard.append(jaccard)

    return Dice,Precsion,Jaccard
    

def batch_pix_accuracy(pred,label,nclass=1):
    if nclass==1:
        pred=torch.round(torch.sigmoid(pred)).int()
        pred=pred.cpu().numpy()
    else:
        pred=torch.max(pred,dim=1)
        pred=pred.cpu().numpy()
    label=label.cpu().numpy()
    pixel_labeled = np.sum(label >=0)
    pixel_correct=np.sum(pred==label)
    
    assert pixel_correct <= pixel_labeled, \
        "Correct area should be smaller than Labeled"
    
    return pixel_correct,pixel_labeled

def batch_intersection_union(predict, target, nclass):

    """Batch Intersection of Union
    Args:
        predict: input 4D tensor
        target: label 3D tensor
        nclass: number of categories (int),note: not include background
    """
    if nclass==1:
        pred=torch.round(torch.sigmoid(predict)).int()
        pred=pred.cpu().numpy()
        target = target.cpu().numpy()
        area_inter=np.sum(pred*target)
        area_union=np.sum(pred)+np.sum(target)-area_inter

        return area_inter,area_union

In [ ]:
import torch.utils.model_zoo as model_zoo
import torchsummary


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride


    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)


        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * Bottleneck.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * Bottleneck.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, deep_base=False, stem_width=32):
        super(ResNet, self).__init__()
        self.inplanes = stem_width*2 if deep_base else 64
        if deep_base:
            self.conv1= nn.Sequential(
                nn.Conv2d(3, stem_width, kernel_size=3, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(stem_width),
                nn.ReLU(inplace=True),
                nn.Conv2d(stem_width, stem_width, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(stem_width),
                nn.ReLU(inplace=True),
                nn.Conv2d(stem_width, stem_width*2, kernel_size=3, stride=1, padding=1, bias=False),
            )
        else:
            self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                   bias=False)
        
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        t = self.layer4(x)

        # x = self.avgpool(x)
        # x = x.view(x.size(0), -1)
        # x = self.fc(x)

        return t
def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    model_dict = model.state_dict()


    if pretrained:
        pretrained_dict=model_zoo.load_url(model_urls['resnet34'],model_dir='../pretrained')# Modify 'model_dir' according to your own path
        print('Petrain Model Have been loaded!')
        pretrained_dict =  {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)

        model.load_state_dict(model_dict)
    return model

In [ ]:
up_kwargs = {'mode': 'bilinear', 'align_corners': True}
class GPG_3(nn.Module):
    def __init__(self, in_channels, width=512, up_kwargs=None,norm_layer=nn.BatchNorm2d):
        super(GPG_3, self).__init__()
        self.up_kwargs = up_kwargs
        

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels[-1], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels[-2], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels[-3], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv_out = nn.Sequential(
            nn.Conv2d(3*width, width, 1, padding=0, bias=False),
            nn.BatchNorm2d(width))
        
        self.dilation1 = nn.Sequential(SeparableConv2d(3*width, width, kernel_size=3, padding=1, dilation=1, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        self.dilation2 = nn.Sequential(SeparableConv2d(3*width, width, kernel_size=3, padding=2, dilation=2, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        self.dilation3 = nn.Sequential(SeparableConv2d(3*width, width, kernel_size=3, padding=4, dilation=4, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                init.normal_(m.weight.data, 1.0, 0.02)
                init.constant_(m.bias.data, 0.0)

    def forward(self, *inputs):
        feats = [self.conv5(inputs[-1]), self.conv4(inputs[-2]), self.conv3(inputs[-3])]
        _, _, h, w = feats[-1].size()
        feats[-2] = F.interpolate(feats[-2], (h, w), **self.up_kwargs)
        feats[-3] = F.interpolate(feats[-3], (h, w), **self.up_kwargs)
        feat = torch.cat(feats, dim=1)
        feat = torch.cat([self.dilation1(feat), self.dilation2(feat), self.dilation3(feat)], dim=1)
        feat=self.conv_out(feat)
        return feat
class GPG_4(nn.Module):
    def __init__(self, in_channels, width=512, up_kwargs=None,norm_layer=nn.BatchNorm2d):
        super(GPG_4, self).__init__()
        self.up_kwargs = up_kwargs
        

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels[-1], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels[-2], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv_out = nn.Sequential(
            nn.Conv2d(2*width, width, 1, padding=0, bias=False),
            nn.BatchNorm2d(width))
        
        self.dilation1 = nn.Sequential(SeparableConv2d(2*width, width, kernel_size=3, padding=1, dilation=1, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        self.dilation2 = nn.Sequential(SeparableConv2d(2*width, width, kernel_size=3, padding=2, dilation=2, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                init.normal_(m.weight.data, 1.0, 0.02)
                init.constant_(m.bias.data, 0.0)

    def forward(self, *inputs):

        feats = [self.conv5(inputs[-1]), self.conv4(inputs[-2])]
        _, _, h, w = feats[-1].size()
        feats[-2] = F.interpolate(feats[-2], (h, w), **self.up_kwargs)
        feat = torch.cat(feats, dim=1)
        feat = torch.cat([self.dilation1(feat), self.dilation2(feat)], dim=1)
        feat=self.conv_out(feat)
        return feat
class GPG_2(nn.Module):
    def __init__(self, in_channels, width=512, up_kwargs=None,norm_layer=nn.BatchNorm2d):
        super(GPG_2, self).__init__()
        self.up_kwargs = up_kwargs
        

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels[-1], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels[-2], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels[-3], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels[-4], width, 3, padding=1, bias=False),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True))

        self.conv_out = nn.Sequential(
            nn.Conv2d(4*width, width, 1, padding=0, bias=False),
            nn.BatchNorm2d(width))
        
        self.dilation1 = nn.Sequential(SeparableConv2d(4*width, width, kernel_size=3, padding=1, dilation=1, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        self.dilation2 = nn.Sequential(SeparableConv2d(4*width, width, kernel_size=3, padding=2, dilation=2, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        self.dilation3 = nn.Sequential(SeparableConv2d(4*width, width, kernel_size=3, padding=4, dilation=4, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        self.dilation4 = nn.Sequential(SeparableConv2d(4*width, width, kernel_size=3, padding=8, dilation=8, bias=False),
                                       nn.BatchNorm2d(width),
                                       nn.ReLU(inplace=True))
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                init.normal_(m.weight.data, 1.0, 0.02)
                init.constant_(m.bias.data, 0.0)

    def forward(self, *inputs):

        feats = [self.conv5(inputs[-1]), self.conv4(inputs[-2]),self.conv3(inputs[-3]),self.conv2(inputs[-4])]
        _, _, h, w = feats[-1].size()
        feats[-2] = F.interpolate(feats[-2], (h, w), **self.up_kwargs)
        feats[-3] = F.interpolate(feats[-3], (h, w), **self.up_kwargs)
        feats[-4] = F.interpolate(feats[-4], (h, w), **self.up_kwargs)
        feat = torch.cat(feats, dim=1)
        feat = torch.cat([self.dilation1(feat), self.dilation2(feat), self.dilation3(feat), self.dilation4(feat)], dim=1)
        feat=self.conv_out(feat)
        return feat

class BaseNetHead(nn.Module):
    def __init__(self, in_planes, out_planes, scale,
                 is_aux=False, norm_layer=nn.BatchNorm2d):
        super(BaseNetHead, self).__init__()
        if is_aux:
            self.conv_1x1_3x3=nn.Sequential(
                ConvBnRelu(in_planes, 64, 1, 1, 0,
                                       has_bn=True, norm_layer=norm_layer,
                                       has_relu=True, has_bias=False),
                ConvBnRelu(64, 64, 3, 1, 1,
                                       has_bn=True, norm_layer=norm_layer,
                                       has_relu=True, has_bias=False))
        else:
            self.conv_1x1_3x3=nn.Sequential(
                ConvBnRelu(in_planes, 32, 1, 1, 0,
                                       has_bn=True, norm_layer=norm_layer,
                                       has_relu=True, has_bias=False),
                ConvBnRelu(32, 32, 3, 1, 1,
                                       has_bn=True, norm_layer=norm_layer,
                                       has_relu=True, has_bias=False))
        # self.dropout = nn.Dropout(0.1)
        if is_aux:
            self.conv_1x1_2 = nn.Conv2d(64, out_planes, kernel_size=1,
                                      stride=1, padding=0)
        else:
            self.conv_1x1_2 = nn.Conv2d(32, out_planes, kernel_size=1,
                                      stride=1, padding=0)
        self.scale = scale
            
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                init.normal_(m.weight.data, 1.0, 0.02)
                init.constant_(m.bias.data, 0.0)

    def forward(self, x):

        if self.scale > 1:
            x = F.interpolate(x, scale_factor=self.scale,
                                   mode='bilinear',
                                   align_corners=True)
        fm = self.conv_1x1_3x3(x)
        # fm = self.dropout(fm)
        output = self.conv_1x1_2(fm)
        return output
class SAPblock(nn.Module):
    def __init__(self, in_channels):
        super(SAPblock, self).__init__()
        self.conv3x3=nn.Conv2d(in_channels=in_channels, out_channels=in_channels,dilation=1,kernel_size=3, padding=1)
        
        self.bn=nn.ModuleList([nn.BatchNorm2d(in_channels),nn.BatchNorm2d(in_channels),nn.BatchNorm2d(in_channels)]) 
        self.conv1x1=nn.ModuleList([nn.Conv2d(in_channels=2*in_channels, out_channels=in_channels,dilation=1,kernel_size=1, padding=0),
                                    nn.Conv2d(in_channels=2*in_channels, out_channels=in_channels,dilation=1,kernel_size=1, padding=0)])
        self.conv3x3_1=nn.ModuleList([nn.Conv2d(in_channels=in_channels, out_channels=in_channels//2,dilation=1,kernel_size=3, padding=1),
                                      nn.Conv2d(in_channels=in_channels, out_channels=in_channels//2,dilation=1,kernel_size=3, padding=1)])
        self.conv3x3_2=nn.ModuleList([nn.Conv2d(in_channels=in_channels//2, out_channels=2,dilation=1,kernel_size=3, padding=1),
                                      nn.Conv2d(in_channels=in_channels//2, out_channels=2,dilation=1,kernel_size=3, padding=1)])
        self.conv_last=ConvBnRelu(in_planes=in_channels,out_planes=in_channels,ksize=1,stride=1,pad=0,dilation=1)



        self.gamma = nn.Parameter(torch.zeros(1))
    
        self.relu=nn.ReLU(inplace=True)

    def forward(self, x):

        x_size= x.size()

        branches_1=self.conv3x3(x)
        branches_1=self.bn[0](branches_1)

        branches_2=F.conv2d(x,self.conv3x3.weight,padding=2,dilation=2)#share weight
        branches_2=self.bn[1](branches_2)

        branches_3=F.conv2d(x,self.conv3x3.weight,padding=4,dilation=4)#share weight
        branches_3=self.bn[2](branches_3)

        feat=torch.cat([branches_1,branches_2],dim=1)
        # feat=feat_cat.detach()
        feat=self.relu(self.conv1x1[0](feat))
        feat=self.relu(self.conv3x3_1[0](feat))
        att=self.conv3x3_2[0](feat)
        att = F.softmax(att, dim=1)
        
        att_1=att[:,0,:,:].unsqueeze(1)
        att_2=att[:,1,:,:].unsqueeze(1)

        fusion_1_2=att_1*branches_1+att_2*branches_2



        feat1=torch.cat([fusion_1_2,branches_3],dim=1)
        # feat=feat_cat.detach()
        feat1=self.relu(self.conv1x1[0](feat1))
        feat1=self.relu(self.conv3x3_1[0](feat1))
        att1=self.conv3x3_2[0](feat1)
        att1 = F.softmax(att1, dim=1)
        
        att_1_2=att1[:,0,:,:].unsqueeze(1)
        att_3=att1[:,1,:,:].unsqueeze(1)


        ax=self.relu(self.gamma*(att_1_2*fusion_1_2+att_3*branches_3)+(1-self.gamma)*x)
        ax=self.conv_last(ax)

        return ax
class DecoderBlock(nn.Module):
    def __init__(self, in_planes, out_planes,
                 norm_layer=nn.BatchNorm2d,scale=2,relu=True,last=False):
        super(DecoderBlock, self).__init__()
       

        self.conv_3x3 = ConvBnRelu(in_planes, in_planes, 3, 1, 1,
                                   has_bn=True, norm_layer=norm_layer,
                                   has_relu=True, has_bias=False)
        self.conv_1x1 = ConvBnRelu(in_planes, out_planes, 1, 1, 0,
                                   has_bn=True, norm_layer=norm_layer,
                                   has_relu=True, has_bias=False)
       
        self.sap=SAPblock(in_planes)
        self.scale=scale
        self.last=last

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                init.normal_(m.weight.data, 1.0, 0.02)
                init.constant_(m.bias.data, 0.0)

    def forward(self, x):

        if self.last==False:
            x = self.conv_3x3(x)
            # x=self.sap(x)
        if self.scale>1:
            x=F.interpolate(x,scale_factor=self.scale,mode='bilinear',align_corners=True)
        x=self.conv_1x1(x)
        return x

class SeparableConv2d(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=3, stride=1, padding=1, dilation=1, bias=False, BatchNorm=nn.BatchNorm2d):
        super(SeparableConv2d, self).__init__()

        self.conv1 = nn.Conv2d(inplanes, inplanes, kernel_size, stride, padding, dilation, groups=inplanes, bias=bias)
        self.bn = BatchNorm(inplanes)
        self.pointwise = nn.Conv2d(inplanes, planes, 1, 1, 0, 1, 1, bias=bias)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.pointwise(x)
        return x
    
class ConvBnRelu(nn.Module):
    def __init__(self, in_planes, out_planes, ksize, stride, pad, dilation=1,
                 groups=1, has_bn=True, norm_layer=nn.BatchNorm2d,
                 has_relu=True, inplace=True, has_bias=False):
        super(ConvBnRelu, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=ksize,
                              stride=stride, padding=pad,
                              dilation=dilation, groups=groups, bias=has_bias)
        self.has_bn = has_bn
        if self.has_bn:
            self.bn = nn.BatchNorm2d(out_planes)
        self.has_relu = has_relu
        if self.has_relu:
            self.relu = nn.ReLU(inplace=inplace)

    def forward(self, x):
        x = self.conv(x)
        if self.has_bn:
            x = self.bn(x)
        if self.has_relu:
            x = self.relu(x)

        return x
    
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        """Global average pooling over the input's spatial dimensions"""
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, inputs):
        in_size = inputs.size()
        inputs = inputs.view((in_size[0], in_size[1], -1)).mean(dim=2)
        inputs = inputs.view(in_size[0], in_size[1], 1, 1)

        return inputs
class CPFNet(nn.Module):
    def __init__(self,out_planes=1,ccm=True,norm_layer=nn.BatchNorm2d,is_training=True,expansion=2,base_channel=32):
        super(CPFNet,self).__init__()
        
        self.backbone =resnet34(pretrained=False)
        self.expansion=expansion
        self.base_channel=base_channel
        if self.expansion==4 and self.base_channel==64:
            expan=[512,1024,2048]
            spatial_ch=[128,256]
        elif self.expansion==4 and self.base_channel==32:
            expan=[256,512,1024]
            spatial_ch=[32,128]
            conv_channel_up=[256,384,512]
        elif self.expansion==2 and self.base_channel==32:
            expan=[128,256,512]
            spatial_ch=[64,64]
            conv_channel_up=[128,256,512]
      
        conv_channel = expan[0]
        self.is_training = is_training
        self.sap=SAPblock(expan[-1])

        self.decoder5=DecoderBlock(expan[-1],expan[-2],relu=False,last=True) #256
        self.decoder4=DecoderBlock(expan[-2],expan[-3],relu=False) #128
        self.decoder3=DecoderBlock(expan[-3],spatial_ch[-1],relu=False) #64
        self.decoder2=DecoderBlock(spatial_ch[-1],spatial_ch[-2]) #32
        self.mce_2=GPG_2([spatial_ch[-1],expan[0], expan[1], expan[2]],width=spatial_ch[-1], up_kwargs=up_kwargs)
        self.mce_3=GPG_3([expan[0], expan[1], expan[2]],width=expan[0], up_kwargs=up_kwargs)
        self.mce_4=GPG_4([expan[1], expan[2]],width=expan[1], up_kwargs=up_kwargs)

        self.main_head= BaseNetHead(spatial_ch[0], out_planes, 2,
                             is_aux=False, norm_layer=norm_layer)
       
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        c1 = self.backbone.relu(x)#1/2  64
        x = self.backbone.maxpool(c1)
        c2 = self.backbone.layer1(x)#1/4   64
        c3 = self.backbone.layer2(c2)#1/8   128
        c4 = self.backbone.layer3(c3)#1/16   256
        c5 = self.backbone.layer4(c4)#1/32   512
        m2=self.mce_2(c2,c3,c4,c5)
        m3=self.mce_3(c3,c4,c5)
        m4=self.mce_4(c4,c5)
        c5=self.sap(c5)
        d4=self.relu(self.decoder5(c5)+m4)  #256
        d3=self.relu(self.decoder4(d4)+m3)  #128
        d2=self.relu(self.decoder3(d3)+m2) #64
        d1=self.decoder2(d2)+c1 #32
        main_out=self.main_head(d1)
        main_out=F.log_softmax(main_out, dim=1)
        #print("main_out.size()")
        #print(main_out.size())torch.Size([2, 2, 256, 448])
        return main_out, main_out
    
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                init.normal_(m.weight.data, 1.0, 0.02)
                init.constant_(m.bias.data, 0.0)

### 模型训练

In [ ]:
def val(args, model, dataloader):
    print('\n')
    print('Start Validation!')
    with torch.no_grad():
        model.eval()
        tbar = tqdm.tqdm(dataloader, desc='\r')
       

        total_Dice=[]
        total_Dice1=[]
        total_Dice2=[]
        total_Dice3=[]
        total_Dice.append(total_Dice1)
        total_Dice.append(total_Dice2)
        total_Dice.append(total_Dice3)
        Acc=[]
        
        cur_cube=[]
        cur_label_cube=[]
        next_cube=[]
        counter=0
        end_flag=False

        for i, (data, labels) in enumerate(tbar):
            # tbar.update()
            if torch.cuda.is_available() and args.use_gpu:
                data = data.cuda()
                label = labels[0].cuda()
            slice_num=labels[1][0].long().item()
            
            # get RGB predict image
            
            aux_predict,predicts = model(data)
            
            predict=torch.argmax(torch.exp(predicts),dim=1)
            batch_size=predict.size()[0]

            counter+=batch_size
            if counter<=slice_num:
                cur_cube.append(predict)
                cur_label_cube.append(label)
                if counter==slice_num:
                    end_flag=True
                    counter=0
            else:
                last=batch_size-(counter-slice_num)

                last_p=predict[0:last]
                last_l=label[0:last]

                first_p=predict[last:]
                first_l=label[last:]

                cur_cube.append(last_p)
                cur_label_cube.append(last_l)
                end_flag=True
                counter=counter-slice_num

            if end_flag:
                end_flag=False
                predict_cube=torch.stack(cur_cube,dim=0).squeeze()
                label_cube=torch.stack(cur_label_cube,dim=0).squeeze()
                cur_cube=[]
                cur_label_cube=[]
                if counter!=0:
                    cur_cube.append(first_p)
                    cur_label_cube.append(first_l)

                assert predict_cube.size()[0]==slice_num
                Dice,true_label,acc, mean_dice = eval_multi_seg(predict_cube,label_cube,args.num_classes)

                for class_id in range(args.num_classes-1):
                    if true_label[class_id]!=0:
                        total_Dice[class_id].append(Dice[class_id])
                Acc.append(acc)
                len0=len(total_Dice[0]) if len(total_Dice[0])!=0 else 1
                len1=len(total_Dice[1]) if len(total_Dice[1])!=0 else 1
                len2=len(total_Dice[2]) if len(total_Dice[2])!=0 else 1

                dice1=sum(total_Dice[0])/len0
                dice2=sum(total_Dice[1])/len1
                dice3=sum(total_Dice[2])/len2
                ACC=sum(Acc)/len(Acc)
                tbar.set_description('Mean_D: %3f, Dice1: %.3f, Dice2: %.3f, Dice3: %.3f, ACC: %.3f' % (mean_dice,dice1,dice2,dice3,ACC))
        print('Mean_Dice:', mean_dice)
        print('Dice1:',dice1)
        print('Dice2:',dice2)
        print('Dice3:',dice3)
        print('Acc:',ACC)
        return mean_dice,dice1,dice2,dice3,ACC


def train(args, model, optimizer,criterion, dataloader_train, dataloader_val,):
    current_time = datetime.now().strftime('%b%d_%H-%M-%S')
    log_dir = os.path.join(args.log_dirs, current_time + '_' + socket.gethostname())
    writer = SummaryWriter(log_dir=log_dir)
    step = 0
    best_pred=0.0
    for epoch in range(args.num_epochs):
        lr = adjust_learning_rate(args,optimizer,epoch) 
        model.train()
        tq = tqdm.tqdm(total=len(dataloader_train) * args.batch_size)
        tq.set_description('checkepoch %d, lr %f' % (epoch, lr))
        loss_record = []
        train_loss=0.0
#        is_best=False
        print(list(enumerate(dataloader_train)))
        for i,(data, label) in enumerate(dataloader_train):
            # if i>9:
            #     break
            #print("there")
            if torch.cuda.is_available() and args.use_gpu:
                data = data.cuda()
                label = label.cuda().long()
            optimizer.zero_grad()
            #print("here")
            aux_out,main_out = model(data)
            loss_aux=F.nll_loss(aux_out,label,weight=None)
            loss_main= criterion[1](main_out, label)
            loss =loss_aux+loss_main
            loss.backward()
            optimizer.step()
            tq.update(args.batch_size)
            train_loss += loss.item()
            tq.set_postfix(loss='%.6f' % (train_loss/(i+1)))
            step += 1
            if step%10==0:
                writer.add_scalar('Train/loss_step', loss, step)
            loss_record.append(loss.item())
        tq.close()
        loss_train_mean = np.mean(loss_record)
        writer.add_scalar('Train/loss_epoch', float(loss_train_mean), epoch)
        print('loss for train : %f' % (loss_train_mean))
        

        if epoch % args.validation_step == 0:
            mean_Dice,Dice1,Dice2,Dice3,acc= val(args, model, dataloader_val)
            writer.add_scalar('Valid/Mean_val', mean_Dice, epoch)
            writer.add_scalar('Valid/Dice1_val', Dice1, epoch)
            writer.add_scalar('Valid/Dice2_val', Dice2, epoch)
            writer.add_scalar('Valid/Dice3_val', Dice3, epoch)
            writer.add_scalar('Valid/Acc_val', acc, epoch)
            # mean_Dice=(Dice1+Dice2+Dice3)/3.0
            is_best=mean_Dice > best_pred
            best_pred = max(best_pred, mean_Dice)
            print(best_pred)
            checkpoint_dir = args.save_model_path
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            checkpoint_latest =os.path.join(checkpoint_dir, 'checkpoint_latest.pth')
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'best_dice': best_pred,
                    }, best_pred,epoch,is_best, checkpoint_dir, filename=checkpoint_latest)
                    
def test(model,dataloader, args, save_path):
    print('start test!')
    with torch.no_grad():
        model.eval()
        # precision_record = []
        tq = tqdm.tqdm(dataloader,desc='\r')
        tq.set_description('test')
        comments=os.getcwd().split('\\')[-1]
        for i, (data, label_path) in enumerate(tq):
            if torch.cuda.is_available() and args.use_gpu:
                data = data.cuda()
                # label = label.cuda()
            aux_pred,predict = model(data)
            predict=torch.argmax(torch.exp(predict),dim=1)
            pred=predict.data.cpu().numpy()
            pred_RGB=Data.COLOR_DICT[pred.astype(np.uint8)]
            
            for index,item in enumerate(label_path):
                img=Image.fromarray(pred_RGB[index].squeeze().astype(np.uint8))
                _, name = os.path.split(item)

                img.save(os.path.join(save_path, name))
                # tq.set_postfix(str=str(save_img_path))
        tq.close()
            
def main(mode='train',args=None):


    # create dataset and dataloader
    dataset_path = args.data
    dataset_train = Data(os.path.join(dataset_path,'train'), scale=(args.crop_width, args.crop_height),mode='train')
    dataloader_train = DataLoader(
        dataset_train,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
        pin_memory=True
    )
    
    dataset_val = Data(os.path.join(dataset_path, 'val'), scale=(args.crop_height, args.crop_width),mode='val')
    dataloader_val = DataLoader(
        dataset_val,
        # this has to be 1
        batch_size=1,
        shuffle=True,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True
    )
    dataset_test=Data(os.path.join(dataset_path,'test'), scale=(args.crop_height, args.crop_width),mode='test')
    dataloader_test = DataLoader(
        dataset_test,
        # this has to be 1
        batch_size=1,
        shuffle=False,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True
    )
    # build model
    os.environ['CUDA_VISIBLE_DEVICES'] = args.cuda
    
    
    #load model
    model_all={'BaseNet': CPFNet(out_planes=args.num_classes)}
    model=model_all[args.net_work]
    print(args.net_work)
    cudnn.benchmark = True
    if torch.cuda.is_available() and args.use_gpu:
        model = torch.nn.DataParallel(model).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
    criterion_aux=nn.NLLLoss(weight=None)
    criterion_main=Multi_DiceLoss(class_num=args.num_classes)
    criterion=[criterion_aux,criterion_main]
    if mode=='train':
        train(args, model, optimizer,criterion, dataloader_train, dataloader_val)

In [ ]:
if __name__ == '__main__':
    seed=1234
    torch.manual_seed(seed)   # 固定初始化
    torch.cuda.manual_seed_all(seed)
    args=DefaultConfig()
    modes = 'train'
    if modes=='train':
        main(mode='train', args=args)

BaseNet


checkepoch 0, lr 0.010000:   0%|          | 0/216 [00:00<?, ?it/s]

[(0, [tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0235, 0.1333, 0.1922,  ..., 0.1373, 0.1255, 0.1176],
          [0.0157, 0.1059, 0.1765,  ..., 0.1294, 0.1176, 0.0745],
          [0.0118, 0.0980, 0.2000,  ..., 0.1137, 0.0980, 0.0353]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0235, 0.1333, 0.1922,  ..., 0.1373, 0.1255, 0.1176],
          [0.0157, 0.1059, 0.1765,  ..., 0.1294, 0.1176, 0.0745],
          [0.0118, 0.0980, 0.2000,  ..., 0.1137, 0.0980, 0.0353]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  

RuntimeError: ignored

### 预测

In [ ]:
if __name__ == '__main__':
    # 每次结果都单独保存
    save_path = r'/content/drive/MyDrive/Blood/train_dataset/outcome/predict1'
    model_path = r'/content/drive/MyDrive/Blood/model/temp_model/checkpoint_latest.pth'
    dataset_test = Data(os.path.join(DefaultConfig.data, 'test'), scale=(DefaultConfig.crop_width,DefaultConfig.crop_height), mode='test')
    args = DefaultConfig()
    dataloader_test = DataLoader(
        dataset_test,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False
    )
    os.environ['CUDA_VISIBLE_DEVICES'] = args.cuda
    model_all = {'BaseNet': CPFNet(out_planes=args.num_classes)}
    model = model_all[args.net_work]
    cudnn.benchmark = True
    if torch.cuda.is_available() and args.use_gpu:
        model = torch.nn.DataParallel(model).cuda()
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    test(model, dataloader_test, args, save_path)